Dưới đây là đề + code mẫu cho lọc cộng tác kNN (cả user-based & item-based) với bộ dữ liệu nhỏ tự tạo – giải thích kỹ từng bước, đúng tinh thần bạn cần luyện thi hoặc demo trên lớp.

Câu 1: User-based collaborative filtering (kNN)

Dùng k=2 để dự đoán rating user 3 sẽ cho item 3 (user 3 chưa rating item này).

Câu 2: Item-based collaborative filtering (kNN)

Dùng k=2 để dự đoán rating user 2 sẽ cho item 2 (user 2 chưa rating item này).

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# ======= Data =======
data = [
    [1, 1, 5],
    [1, 2, 4],
    [1, 3, 2],
    [2, 1, 4],
    [2, 3, 3],
    [2, 4, 5],
    [3, 1, 2],
    [3, 2, 2],
    [3, 4, 4],
    [4, 2, 5],
    [4, 3, 4],
    [4, 4, 2],
]
df = pd.DataFrame(data, columns=["user_id", "item_id", "rating"])

# Tạo ma trận user-item
user_item = df.pivot(index="user_id", columns="item_id", values="rating")
user_item

item_id,1,2,3,4
user_id,,,,
1,5.0,4.0,2.0,NaN
2,4.0,NaN,3.0,5.0
3,2.0,2.0,NaN,4.0
4,NaN,5.0,4.0,2.0


In [3]:
# ----------------------
# Câu 1: User-based kNN (user 3, item 3)
# ----------------------

target_user = 3
target_item = 3

# Xác định các user đã rating target_item
users = user_item.index.tolist()
items = user_item.columns.tolist()

# Tính độ tương đồng cosine giữa target_user với các user khác (dùng cặp chung)
def cosine_on_common(u, v):
    mask = ~np.logical_or(np.isnan(u), np.isnan(v))
    if mask.sum() == 0:
        return 0
    u1, v1 = u[mask], v[mask]
    if np.linalg.norm(u1) == 0 or np.linalg.norm(v1) == 0:
        return 0
    return np.dot(u1, v1) / (np.linalg.norm(u1) * np.linalg.norm(v1))

sim_scores = []
for u in users:
    if u == target_user:
        continue
    sim = cosine_on_common(
        user_item.loc[target_user].values,
        user_item.loc[u].values
    )
    sim_scores.append((u, sim))

# Lấy 2 user gần nhất (kNN)
top2 = sorted(sim_scores, key=lambda x: x[1], reverse=True)[:2]
# print("Top 2 user giống user 3:", top2)

# Lấy rating của các user này cho item 3 (nếu có)
num = 0
den = 0
for u, sim in top2:
    rating = user_item.loc[u, target_item]
    if not np.isnan(rating):
        num += sim * rating
        den += sim
pred_user_based = num / den if den != 0 else np.nan

print(f"[User-based] Dự đoán rating của user 3 cho item 3: {pred_user_based:.2f}")


[User-based] Dự đoán rating của user 3 cho item 3: 2.50


In [4]:

# ----------------------
# Câu 2: Item-based kNN (user 2, item 2)
# ----------------------

target_user2 = 2
target_item2 = 2

# Tính độ tương đồng giữa item 2 và các item khác
item_vectors = user_item.T.values
item_ids = user_item.columns.tolist()

sim_scores2 = []
for i, item_id in enumerate(item_ids):
    if item_id == target_item2:
        continue
    sim = cosine_on_common(
        user_item[target_item2].values,
        user_item[item_id].values
    )
    sim_scores2.append((item_id, sim))

top2_items = sorted(sim_scores2, key=lambda x: x[1], reverse=True)[:2]
# print("Top 2 items giống item 2:", top2_items)

# Lấy rating user 2 cho các item này
num2 = 0
den2 = 0
for item_id, sim in top2_items:
    rating = user_item.loc[target_user2, item_id]
    if not np.isnan(rating):
        num2 += sim * rating
        den2 += sim

pred_item_based = num2 / den2 if den2 != 0 else np.nan

print(f"[Item-based] Dự đoán rating của user 2 cho item 2: {pred_item_based:.2f}")


[Item-based] Dự đoán rating của user 2 cho item 2: 3.50
